In [3]:
import pandas as pd
import geopandas as gpd
import fiona
import warnings 
warnings.filterwarnings('ignore')

In [4]:
loc = r"H:\27\27381 - Caltrans District Plans\Task 3 Methodology\sliding_wind"

In [28]:
# Load Roadways
hwy = pd.read_csv(loc + r"\D12_HWY_CLEANED_20250805 1.csv")
intx = pd.read_csv(loc + r"\D12_INTX_CLEANED_20250806.csv")
intx = intx[intx['Drop'] != 1]
ramp = pd.read_csv(loc + r"\D12_RAMP_20250716.csv")
ramp = ramp[ramp['Drop'] != 1]

In [29]:
agg_routes = hwy.groupby('PMRouteID_Fix').agg({'begPM_Fix': 'min', 'endPM_Fix': 'max'}).reset_index()

In [30]:
SlidingDist = 2
SInterval = 1

# Create walk through each dissolved road. Information about windows is recorded in lists that
# are later processed into dataframe
seg_ID_lst = []
# road_diss_lst = []
RID_lst = []
BMP_lst = []
EMP_lst = []
BMP_full = []
EMP_full = []

# Index used to ID each window
i = 1

for index, row in agg_routes.iterrows():
    lengthTemp = row['endPM_Fix'] - row['begPM_Fix']
    BMP = row['begPM_Fix']
    EMP = row['endPM_Fix']

    SlidePostStart = BMP
    SlidePostEnd = BMP + SlidingDist

    # If segment is less than the analysis window
    if lengthTemp <= SlidingDist:
        seg_ID_lst.append(i)
        # road_diss_lst.append(row['diss_id'])
        RID_lst.append(row['PMRouteID_Fix'])
        BMP_lst.append(row['begPM_Fix'])
        EMP_lst.append(row['endPM_Fix'])
        BMP_full.append(row['begPM_Fix'])
        EMP_full.append(row['endPM_Fix'])
        i = i + 1

    # If segment is greater than the analysis window but less than the sliding distance plus analysis interval
    # Creates one window from BMP and one working backwards from end of the segment
    elif (lengthTemp > SlidingDist) and (lengthTemp <= (SlidingDist + SInterval)):
        seg_ID_lst.append(i)
        # road_diss_lst.append(row['diss_id'])
        RID_lst.append(row['PMRouteID_Fix'])
        BMP_lst.append(row['begPM_Fix'])
        EMP_lst.append(SlidePostEnd)
        BMP_full.append(row['begPM_Fix'])
        EMP_full.append(row['endPM_Fix'])
        i = i + 1

        SlidePostStart = SlidePostStart + SInterval
        seg_ID_lst.append(i)
        # road_diss_lst.append(row['diss_id'])
        RID_lst.append(row['PMRouteID_Fix'])
        BMP_lst.append(row['endPM_Fix'] - SlidingDist)
        EMP_lst.append(row['endPM_Fix'])
        BMP_full.append(row['begPM_Fix'])
        EMP_full.append(row['endPM_Fix'])
        i = i + 1

    # If the segment is fits at least two analysis windows offset by the sliding distance
    # Create sliding windows while distance and end with last segment working backwards from end of segment
    elif lengthTemp > (SlidingDist + SInterval):
        while SlidePostEnd < EMP:
            seg_ID_lst.append(i)
            # road_diss_lst.append(row['diss_id'])
            RID_lst.append(row['PMRouteID_Fix'])
            BMP_lst.append(SlidePostStart)
            EMP_lst.append(SlidePostEnd)
            BMP_full.append(row['begPM_Fix'])
            EMP_full.append(row['endPM_Fix'])
            i = i + 1
            SlidePostStart = SlidePostStart + SInterval
            SlidePostEnd = SlidePostEnd + SInterval

        seg_ID_lst.append(i)
        # road_diss_lst.append(row['diss_id'])
        RID_lst.append(row['PMRouteID_Fix'])
        BMP_lst.append(row['endPM_Fix'] - SlidingDist)
        EMP_lst.append(row['endPM_Fix'])
        BMP_full.append(row['begPM_Fix'])
        EMP_full.append(row['endPM_Fix'])
        i = i + 1

# Combine lists into a dataframe
d = {'Seg_ID': seg_ID_lst, 'PMRouteID_Fix': RID_lst, 'begPM_Fix': BMP_lst, 'endPM_Fix': EMP_lst, \
        'BMP_full': BMP_full, 'EMP_full': EMP_full}
sl_winds = pd.DataFrame(d)

In [31]:
sl_winds

,Seg_ID,PMRouteID_Fix,begPM_Fix,endPM_Fix,BMP_full,EMP_full
0,1,LA.039.D.L,17.266,18.446,17.266,18.446
1,2,LA.039.D.R,17.266,18.446,17.266,18.446
2,3,LA.405...L,0.000,0.266,0.000,0.266
3,4,LA.405...R,0.000,0.266,0.000,0.266
4,5,LA.605.H.L,0.879,0.955,0.879,0.955
...,...,...,...,...,...,...
707,708,ORA405...R,22.178,24.178,0.230,24.178
708,709,ORA605...L,3.091,3.500,3.091,3.500
709,710,ORA605...R,3.091,3.500,3.091,3.500
710,711,ORA605.R.L,0.000,1.643,0.000,1.643


### Adding weights to raw roadway, ramp, intersection dataframes

In [32]:
# Function to add points to ramp, hwy, road file
def add_risk_points(field1, filt1, field2, filt2, field3, filt3, true_value, false_value, exist_total):
    if (field1 in filt1) & (field2 in filt2) & (field3 in filt3):
        return true_value + exist_total
    else:
        return false_value + exist_total

In [35]:
loc_prior = r"H:\27\27381 - Caltrans District Plans\Task 3 Methodology\sliding_wind"

# Read in csv with data on what is prioritized
intx_priority = pd.read_excel(loc + r"\testbranches.xlsx", sheet_name='intx_points')
ramp_priority = pd.read_excel(loc + r"\testbranches.xlsx", sheet_name='ramp_points')
hwy_priority = pd.read_excel(loc + r"\testbranches.xlsx", sheet_name='hwy_points')


In [33]:
# hwy['points'] = 1
# hwy.apply(lambda row: add_risk_points(row['PlaceType'],['Urban Community'],row['PlaceType'],['Urban Community'],row['PlaceType'],['Urban Community'], 1, 0, row['points']), axis = 1)

In [48]:
# Intersection Weight Assignment - Points should be value for if present at all intersections on Sliding Window (i.e., all intersections on sliding window have)
intx['points'] = 0

for index, branch_row in intx_priority.iterrows():
    filt1_in = branch_row['field1']
    filt1_list = branch_row['filt1'].split(",")
    filt1_in = branch_row['field1']
    filt1_list = branch_row['filt1'].to_list()
    filt1_in = branch_row['field1']
    filt1_list = branch_row['filt1'].to_list()
    # intx.apply(lambda row: add_risk_points(row[branch_row['field1']],branch_row['filt1'],row[branch_row['field2']],branch_row['filt2'],row[branch_row['field3']],branch_row['filt3'], branch_row['true_value'], branch_row['false_value'], row['points']), axis = 1)

AttributeError: 'int' object has no attribute 'split'

In [49]:
filt1_list

['45', ' 55', ' 60']

In [ ]:
# Ramp Weight Assignment - Points should be value for if present at all ramps on Sliding Window (i.e., full segment contains factor)

In [10]:
# Ramp Weight Assignment - Points are value for presence at all ramps on Sliding Window (i.e., all ramps on sliding window have)



### Prep data for joining to sliding windows

In [13]:
intx['mp'] = intx[['begPM_Fix','endPM_Fix']].mean(axis = 1)
ramp['mp'] = ramp[['begPM_Fix','endPM_Fix']].mean(axis = 1)
hwy['start'] = hwy['begPM_Fix']
hwy['end'] = hwy['endPM_Fix']
sl_winds['length'] = (sl_winds['endPM_Fix'] - sl_winds['begPM_Fix'])

In [15]:
sl_intx = sl_winds.merge(intx[['PMRouteID_Fix','mp','intx_points']], how = 'left')
sl_intx = sl_intx[(sl_intx['mp'] >= sl_intx['begPM_Fix']) & (sl_intx['mp'] <= sl_intx['endPM_Fix'])]

sl_ramp = sl_winds.merge(ramp[['PMRouteID_Fix','mp','ramp_points']], how = 'left')
sl_ramp = sl_ramp[(sl_ramp['mp'] >= sl_ramp['begPM_Fix']) & (sl_ramp['mp'] <= sl_ramp['endPM_Fix'])]

sl_hwy = sl_winds.merge(hwy[['PMRouteID_Fix','start','end','points']], how = 'left')
sl_hwy['road_points_scale'] = 0
sl_hwy['road_points_scale'][(sl_hwy['start'] <= sl_hwy['begPM_Fix']) & (sl_hwy['end'] >= sl_hwy['endPM_Fix'])] = sl_hwy['points']
sl_hwy['road_points_scale'][(sl_hwy['start'] >= sl_hwy['begPM_Fix']) & (sl_hwy['end'] >= sl_hwy['endPM_Fix'])] = sl_hwy['points'] * ((sl_hwy['endPM_Fix'] - sl_hwy['start'])/SlidingDist)
sl_hwy['road_points_scale'][(sl_hwy['start'] <= sl_hwy['begPM_Fix']) & (sl_hwy['end'] <= sl_hwy['endPM_Fix'])] = sl_hwy['points'] * ((sl_hwy['end'] - sl_hwy['begPM_Fix'])/SlidingDist)
sl_hwy['road_points_scale'][(sl_hwy['end'] <= sl_hwy['begPM_Fix']) | (sl_hwy['start'] >= sl_hwy['endPM_Fix'])] = 0

In [25]:
# Aggregating Ramps - Average ramp score
sl_ramp_sum = sl_winds.merge(sl_ramp.groupby('Seg_ID')['points'].mean(),on = 'Seg_ID')

# Aggregating Intersection - Average ramp score
sl_intx_sum =sl_winds.merge(sl_intx.groupby('Seg_ID')['points'].mean(),on = 'Seg_ID')

# Aggregating Highways
sl_hwy_sum = sl_winds.merge(sl_hwy.groupby('Seg_ID')['points'].sum(),on = 'Seg_ID')

,Seg_ID,PMRouteID_Fix,begPM_Fix,endPM_Fix,BMP_full,EMP_full,length,road_points_scale
0,1,LA.039.D.L,17.266,18.446,17.266,18.446,0.5900,0.037005
1,2,LA.039.D.R,17.266,18.446,17.266,18.446,0.5900,0.037005
2,3,LA.405...L,0.000,0.266,0.000,0.266,0.1330,0.009005
3,4,LA.405...R,0.000,0.266,0.000,0.266,0.1330,0.009005
4,5,LA.605.H.L,0.879,0.955,0.879,0.955,0.0380,0.076000
...,...,...,...,...,...,...,...,...
707,708,ORA405...R,22.178,24.178,0.230,24.178,1.0000,0.017505
708,709,ORA605...L,3.091,3.500,3.091,3.500,0.2045,0.009000
709,710,ORA605...R,3.091,3.500,3.091,3.500,0.2045,0.009000
710,711,ORA605.R.L,0.000,1.643,0.000,1.643,0.8215,0.026010


### Final Cleaning to Confirm if Segment is REAL in Caltrans